In [1]:
import numpy as np
from pyscf import gto, scf, fci
from multiprocessing.pool import Pool
from numba import prange, jit, njit
#from numba.openmp import openmp_context as openmp

In [2]:
def expand_matrix(P):
    Paa = P
    Pbb = P
    Pab = np.zeros(P.shape)
    Pba = np.zeros(P.shape)
    PE = np.concatenate((np.concatenate((Paa, Pab), axis=1), np.concatenate((Pba, Pbb), axis=1)), axis=0) 
    return PE

In [3]:
# for jit and prange
@jit(nopython=True, parallel=True)
def ONERDMFT_hartree_energy_parallel(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  prange(0,M):
        for b in range(0,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[a]*n[b]*C[mu,a]*C[nu,a]*C[kappa,b]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]

    return 1/2.*energy

In [4]:
# for spinorbitals, the Umrigar Hartree term is a Hartree term from which the self interaction has been removed 
@jit(nopython=True, parallel=True)
def ONERDMFT_Umrigar_hartree_energy_parallel(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  prange(0,M):
        for b in [b for b in range(0,M) if b!=a ]:
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[a]*n[b]*C[mu,a]*C[nu,a]*C[kappa,b]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]

    return 1/2.*energy


In [5]:
@jit(nopython=True, parallel=True)
def ONERDMFT_Umrigar_exchange_correlation_energy_parallel(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  prange(0,M//2):
        for b in [b for b in range(0,M//2) if b!=a ]:
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += np.sqrt(n[a]*n[b])*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]
    for a in  prange(M//2,M):
        for b in [b for b in range(M//2,M) if b!=a ]:
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += np.sqrt(n[a]*n[b])*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]

    

    return -1/2.*energy

In [6]:
@jit(parallel=True)
def ONERDMFT_exchange_energy_parallel(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  prange(0,M//2):
        for b in range(0,M//2):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[a]*n[b]*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]
    for a in  prange(M//2,M):
        for b in range(M//2,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[a]*n[b]*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]


    return -1/2.*energy

In [7]:
@jit(parallel=True)
def ONERDMFT_Mueller_exchange_correlation_energy_parallel(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  prange(0,M//2):
        for b in range(0,M//2):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += np.sqrt(n[a]*n[b])*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]
    for a in  prange(M//2,M):
        for b in range(M//2,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += np.sqrt(n[a]*n[b])*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]


    return -1/2.*energy

In [8]:
mol = gto.Mole()
mol.atom = """
    Li    0.    0.    0.
"""
# this basis has 2 functions for Helium
#mol.basis = "6-31g" 
#mol.basis = "ccpvdz"
mol.basis = "sto-6g"
mol.spin=1
mol.build()

# the 2 electron integrals \langle \mu \nu | \kappa \lambda \rangle have M^4 entries
eri = mol.intor('int2e')
S = mol.intor('int1e_ovlp')

print(eri.shape)

## Run Hartree-Fock.
mf = scf.UHF(mol)
mf.kernel()



(5, 5, 5, 5)
converged SCF energy = -7.39993122985156  <S^2> = 0.75  2S+1 = 2


np.float64(-7.399931229851557)

In [9]:
C = mf.mo_coeff
h = mf.get_hcore()
N_a, N_b = mol.nelec
M = eri.shape[0]
print(C.shape)


P_aa=np.matmul(C[0,:,0:N_a],C[0,:,0:N_a].T)
P_bb=np.matmul(C[0,:,0:N_b],C[0,:,0:N_b].T)




(2, 5, 5)


In [10]:
P = mf.make_rdm1()
print(P.shape)

(2, 5, 5)


In [11]:
P_aa = P[0,:,:]
P_bb = P[1,:,:]

In [12]:
gamma, occu, naturalC = {},{},{}
for i, name in enumerate(['alpha', 'beta']):
    gamma[name] = np.matmul(np.matmul(C[i,:,:].T,np.matmul(np.matmul(S,P[i,:,:]),S)), C[i,:,:])
    occu[name], naturalC[name] = np.linalg.eigh(gamma[name])

In [13]:
# need to first make gamma and then expand into blockdiagonal so "fake" alpha and "beta" are seperated in this fashion
CE = expand_matrix(C[0,:,:])
CE[M:,M:]=C[1,:,:]
occu = np.append(occu['alpha'], occu['beta'])
naturalCE = expand_matrix(naturalC['alpha'])
naturalCE[M:,M:] = naturalC['beta']
naturalCTT = np.matmul(CE,naturalCE)

In [14]:
for i, n  in enumerate(occu):
    if n < 0:
        occu[i] = 0 
print(occu)

[0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00
 1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 5.43407302e-18 1.00000000e+00]


In [15]:
#print(naturalCTT)
E_H = ONERDMFT_hartree_energy_parallel(eri, naturalCTT, occu)
E_U = ONERDMFT_Umrigar_hartree_energy_parallel(eri, naturalCTT, occu)
E_x = ONERDMFT_exchange_energy_parallel(eri, naturalCTT, occu)
E_xc = ONERDMFT_Umrigar_exchange_correlation_energy_parallel(eri, naturalCTT, occu)
print(f"Based on expanded HF density matrix:\nHartree Energy: {E_H}")
print(f"Exchange Energy: {E_x}\nHartree + Exchange {E_H+E_x}")
print(f"Umrigar Hartree: {E_U}\nUmrigar Exchangecorrelation: {E_xc}\nUmrigar Hartree + Exchangecorrelation: {E_U + E_xc}")

/home/jerhard/1RDMFT/lib/python3.12/site-packages/numba/parfors/parfor.py:2395: NumbaPerformanceWarning: 
prange or pndindex loop will not be executed in parallel due to there being more than one entry to or exit from the loop (e.g., an assertion).

File "../../../../tmp/ipykernel_57044/671129904.py", line 7:
<source missing, REPL/exec in use?>

  warnings.warn(
/home/jerhard/1RDMFT/lib/python3.12/site-packages/numba/core/typed_passes.py:336: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see https://numba.readthedocs.io/en/stable/user/parallel.html#diagnostics for help.

File "../../../../tmp/ipykernel_57044/671129904.py", line 2:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaPerformanceWarning(msg,
/home/jerhard/1RDMFT/lib/python3.12/site-packages/numba/parfors/parfor.py:2395: NumbaPerformanceWarning: 
prange or pndindex lo

Based on expanded HF density matrix:
Hartree Energy: 4.283183744168103
Exchange Energy: -1.8390021481362688
Hartree + Exchange 2.4441815960318345
Umrigar Hartree: 2.4754792996540163
Umrigar Exchangecorrelation: -0.031297703699081136
Umrigar Hartree + Exchangecorrelation: 2.444181595954935


In [16]:
FCI = True
if FCI:
    fs = fci.FCI(mol, mf.mo_coeff)
    e, ci = fs.kernel(verbose=0)
FCIgamma_aa, FCIgamma_bb = fci.direct_spin1.make_rdm1s(ci, M, mol.nelec)
print(FCIgamma_aa.shape, FCIgamma_bb.shape)

(5, 5) (5, 5)


In [17]:
FCIoccu_aa, FCInaturalC_aa = np.linalg.eigh(FCIgamma_aa)
FCIoccu_bb, FCInaturalC_bb = np.linalg.eigh(FCIgamma_bb)

FCInaturalCTT_aa = np.matmul(C[0,:,:],FCInaturalC_aa)
FCInaturalCTT_bb = np.matmul(C[1,:,:],FCInaturalC_bb)


FCInaturalCTTE = expand_matrix(FCInaturalCTT_aa)
FCInaturalCTTE[M:,M:] = FCInaturalCTT_bb
FCIoccuE = np.append(FCIoccu_aa,FCIoccu_bb)
#print(FCInaturalCTTE)

In [18]:
print(FCIoccuE)

[ 4.03000085e-05  4.03000085e-05  4.03000085e-05  9.99879100e-01
  1.00000000e+00 -1.23092377e-25  4.03000085e-05  4.03000085e-05
  4.03000085e-05  9.99879100e-01]


In [19]:
for i, n  in enumerate(FCIoccuE):
    if n < 0:
        print(f"{i}-th value is nonsense: {n} and removed.")
        FCIoccuE[i] = 0 
print(FCIoccuE)

5-th value is nonsense: -1.230923765364894e-25 and removed.
[4.03000085e-05 4.03000085e-05 4.03000085e-05 9.99879100e-01
 1.00000000e+00 0.00000000e+00 4.03000085e-05 4.03000085e-05
 4.03000085e-05 9.99879100e-01]


In [20]:
print(f"E_FCI {e} E_HF {mf.e_tot} E_c {e-mf.e_tot}")

E_FCI -7.400238382274363 E_HF -7.399931229851557 E_c -0.0003071524228062117


In [21]:
E_H = ONERDMFT_hartree_energy_parallel(eri, FCInaturalCTTE, FCIoccuE)
E_U = ONERDMFT_Umrigar_hartree_energy_parallel(eri, FCInaturalCTTE, FCIoccuE)
E_x = ONERDMFT_exchange_energy_parallel(eri, FCInaturalCTTE, FCIoccuE)
E_xc = ONERDMFT_Umrigar_exchange_correlation_energy_parallel(eri, FCInaturalCTTE, FCIoccuE)
# E_c = (2*np.trace(np.matmul(h,np.matmul(C, np.matmul(FCIgamma,C.T)))) + E_U + E_xc) - mf.e_tot
print(f"Based on expanded FCI density matrix:\nHartree Energy: {E_H}")
print(f"Exchange Energy: {E_x}\nHartree + Exchange {E_H+E_x}")
print(f"Umrigar Hartree: {E_U}")
print(f"Umrigar Exchange-correlation: {E_xc}\nUmrigar Hartree + Exchange-correlation: {E_U + E_xc}")
# print(f"Umrigar corelation energy {E_c}")

Based on expanded FCI density matrix:
Hartree Energy: 4.282422289202717
Exchange Energy: -1.8385589546104075
Hartree + Exchange 2.4438633345923098
Umrigar Hartree: 2.4821027271175224
Umrigar Exchange-correlation: -0.03977809600477905
Umrigar Hartree + Exchange-correlation: 2.4423246311127436


In [22]:
h = mf.get_hcore()
np.trace(np.matmul(h,np.matmul(C[1,:,:], np.matmul(FCIgamma_bb,C[1,:,:].T))))+np.trace(np.matmul(h,np.matmul(C[0,:,:], np.matmul(FCIgamma_aa,C[0,:,:].T))))+E_U+E_xc

np.float64(-7.401096152964811)